# 随机抽样

In [149]:
import tensorflow as tf
import numpy as np
import random

In [133]:
GEN_RANK_NUM = 4+1
LENGTH = 7
batch_size = 3
K = 2

In [138]:
a = np.random.random_sample((batch_size, LENGTH,GEN_RANK_NUM))
#a = np.reshape(a,[3,70*5])
tensor_a = tf.convert_to_tensor(a)

In [139]:
a.shape

(3, 7, 5)

In [148]:
sess.run(aa[:][0])


array([[1, 1],
       [1, 0]])

In [63]:
tensor_a.get_shape().as_list()[0]

3

In [214]:
sess = tf.Session()
init_op = tf.global_variables_initializer()
sess.run(init_op)

In [115]:
aa=tf.constant([[[1,1],[1,0]],[[0,0],[1,1]]])
bb=tf.constant([[[1,0],[1,0]],[[1,0],[1,0]]])
a = tf.constant(0)
b = tf.constant(0)
aeqb = tf.cast(tf.equal(a,b),tf.int32)
aeqb_int = tf.to_int32(aeqb)
result_0 = tf.equal(a,b)
result_0,aa,bb

(<tf.Tensor 'Equal_27:0' shape=() dtype=bool>,
 <tf.Tensor 'Const_58:0' shape=(2, 2, 2) dtype=int32>,
 <tf.Tensor 'Const_59:0' shape=(2, 2, 2) dtype=int32>)

In [116]:
result = tf.convert_to_tensor(False)
#result = tf.cond(result_0, lambda: tf.concat([aa,bb],axis=0), lambda: bb) 
result = tf.cond(result_0, lambda:True,lambda:result) 
result

<tf.Tensor 'cond_20/Merge:0' shape=() dtype=bool>

In [117]:
c = sess.run(result)
c.shape,c

((), True)

In [90]:
sess.run(aa)

array([[[1, 1],
        [1, 0]],

       [[0, 0],
        [1, 1]]])

In [150]:
t = tf.constant([[[1, 1, 1], [2, 2, 2]],
                 [[3, 3, 3], [4, 4, 4]],
                 [[5, 5, 5], [6, 6, 6]]])
tf.slice(t, [1, 0, 0], [1, 1, 3])  # [[[3, 3, 3]]]
#tf.slice(t, [1, 0, 0], [1, 2, 3])  # [[[3, 3, 3],
                                   #   [4, 4, 4]]]
#tf.slice(t, [1, 0, 0], [2, 1, 3])  # [[[3, 3, 3]],
                                   #  [[5, 5, 5]]]

<tf.Tensor 'Slice_4:0' shape=(1, 1, 3) dtype=int32>

In [120]:
random_shifts = np.random.random([4,70])  # [4,70]
random_shifts /= random_shifts.sum(axis = 1)[:,np.newaxis]
random_shifts

array([[3.15610041e-03, 7.19512116e-03, 5.22508180e-03, 9.52518463e-03,
        1.43609564e-02, 9.40549274e-04, 8.37193730e-03, 6.50926459e-03,
        3.07019520e-02, 3.07323850e-02, 7.73903264e-03, 2.71269860e-02,
        2.02238655e-02, 1.41607958e-02, 1.63389712e-02, 2.63741558e-03,
        2.26440129e-02, 1.84694644e-02, 2.93503052e-02, 1.09395133e-02,
        1.20159463e-02, 3.95270029e-04, 1.64312463e-02, 2.77879375e-03,
        1.60852242e-03, 4.86028485e-03, 1.70136724e-02, 2.92415200e-02,
        1.54949289e-02, 1.58572429e-03, 1.44025215e-02, 1.18506921e-02,
        1.49540229e-02, 1.43198011e-02, 2.39724414e-02, 2.81366652e-02,
        6.58072634e-03, 7.38646714e-03, 6.14210102e-03, 1.34308936e-02,
        2.45154713e-02, 1.67661972e-02, 1.59355058e-02, 2.74204146e-02,
        1.14828590e-02, 4.37905412e-03, 2.86536679e-02, 1.63485151e-03,
        2.98975102e-03, 6.59869029e-03, 6.05938657e-03, 1.85551532e-02,
        1.85973513e-02, 1.16316019e-02, 2.88127358e-03, 1.749106

In [181]:
a = tf.Variable(initial_value=[[0, 0, 0, 0], [0, 0, 0, 0]])
b1 = tf.scatter_update(a, 0, [1, 1, 0, 0])
b2 = tf.scatter_update(a, [1, 1, 1], [[1, 1, 0, 0], [1, 0, 4, 0],[1, 1, 0, 1]])

In [184]:
sess.run(b1)

array([[1, 1, 0, 0],
       [0, 0, 0, 0]])

In [211]:
sess.run(tt)
tt.name


'Variable_9:0'

In [209]:
at = tf.gather(tt, 2)
nr = tf.concat([at[:1], [0]*2], axis=0)
q = tf.scatter_update(tt, 2, nr)
sess.run(q)

array([[1, 1, 1],
       [2, 2, 2],
       [3, 0, 0]])

In [213]:
tt = tf.Variable([[1, 1, 1], [2, 2, 2], [3, 4, 5]])
tt.name


In [218]:
tn = set_value(tt,2,1,3,7)

In [219]:
sess.run(tn)

array([[1, 1, 1],
       [2, 2, 2],
       [0, 7, 7]])

In [212]:
def set_value(matrix, x, y, col_len=4,val=0.):
    # 提取出要更新的行
    row = tf.gather(matrix, x)
    # 构造这行的新数据
    new_row = tf.concat([row[:y], [val]*(col_len-y)], axis=0)
    # 使用 tf.scatter_update 方法进正行替换
    #matrix.assign(tf.scatter_update(matrix, x, new_row))  
    return tf.scatter_update(matrix, x, new_row)

In [134]:
def sample_one(table):
    random_shifts = np.random.random(table.shape)  # [4,70]
    random_shifts /= random_shifts.sum(axis = 1)[:,np.newaxis]
    result = []
    print('sample_one,table_shape',table.shape)
    for i in range(table.shape[0]):  # 每个位置[0,4),一共4个位置，每个位置有70个候选sku
        table[i] /= table[i].sum()
        shifted_probabilities = random_shifts[i] - table[i]
        print('shifted_probabilities.shape:',shifted_probabilities)
        l = np.argpartition(shifted_probabilities, 1, axis=0)[0].astype(np.int64)  # 选出每行最小值的index
        print('argpartition:',shifted_probabilities[l])
        table[i:,l] = 0
        print('table[i:,l],i={},l={}:'.format(i,l),table[i:,l])
        result.append(l)
        print('sample_one',table,l)
    return result

In [ ]:
def tf_sample_one(table):
    random_shifts = np.random.random([LENGTH,GEN_RANK_NUM-1])  # [70,4]
    random_shifts /= random_shifts.sum(axis = 1)[:,np.newaxis]
    result = []
    for m in range(GEN_RANK_NUM-1):  # 4个位置
        table_m = table[:][m]  # [70,1]
        table_m /= tf.reduce_sum(table_m)
        shifted_probabilities = random_shifts[:,i] - table_m
        l = tf.argmin(shifted_probabilities)  # 找出最小的index
        table = set_value(table,x=l,y=m,col_len=GEN_RANK_NUM-1,val=0.)
        result.append(l)
    return result
        

In [135]:
def sample_set(table, n):  # n=K,从70里面抽取20个sku
    result = []
    print('table-shape',table.shape)  # [4,70]
    print('k={}'.format(n))
    for i in range(n):  # 抽样n个list
        r = sample_one(table.copy())  # [1,4]
        print('sample_set,r',r)
        if not r in result:
            result.append(r)  # [20, 4]
    return np.array(result)

In [58]:
def is_comb(result):
    return (np.sum(result) == 6)
def is_right(result):
    return (result[0] == 0 and result[1] == 1 and result[2] == 2 and result[3] == 3)

In [ ]:
def tf_check_repeat(checked, cand_set):
    flag = tf.convert_to_tensor(False)
    for cand_tensor in cand_set:
        flag = tf.cond(checked==cand_tensor, lambda: True, lambda: flag)
    return flag  # 但凡有一个重复则true

In [126]:
def tf_sample_and_test(prob_table, temperature, k=K):
if len(prob_table.shape) == 3:
        prob_table = prob_table[:][:][0:GEN_RANK_NUM-1]
    elif len(prob_table.shape) == 2:
        prob_table = np.reshape(prob_table, (-1, LENGTH, GEN_RANK_NUM))[:][:][0:GEN_RANK_NUM-1]   
    res = []
    prob_table = tf.math.exp(probtable * temperature)  # [bs, LENGTH, GEN_RANK_NUM-1]  [3,70,4]
    
    # 从每条候选概率[70,4]中抽样20个即[20,4]
    for i in range(prob_table.get_shape().as_list()[0]):  # bs
        sample_set = []
        
        while k > 0:  # 抽取20个sku的位置概率
            table  = prob_table[i][LENGTH][GEN_RANK_NUM-1])
            tmp = tf_sample_one(table)  # [70,4] -> [1,4]
            if len(sample_set)==0:
                sample_set_concat = tmp
                k -= 1
            else:
                tmp_len = sample_set_concat.get_shape().as_list()[0]
                tf.cond(check_repeat(tmp,sample_set), 
                        true_fn=sample_set_concat, 
                        false_fn=tf.concat([sample_set_concat,emp],0))  
                if sample_set_concat.get_shape().as_list()[0]!=tmp:
                    sample_set.append(tmp)
                    k -= 1
        res.append(sample_set_concat)
    res = tf.concat(res,0)
    return res  # [bs, K, GEN_RANK_NUM-1]

In [136]:

def sample_and_test(prob_table, temperature, k=K):
    if len(prob_table.shape) == 3:
        prob_table = prob_table[:,:,0:GEN_RANK_NUM-1]
    elif len(prob_table.shape) == 2:
        prob_table = np.reshape(prob_table, (-1, LENGTH, GEN_RANK_NUM))[:,:,0:GEN_RANK_NUM-1]   
    count = 0
    set_count = 0
    right = 0
    comb = 0
    res = []
    for i in range(prob_table.shape[0]):  # 循环batch_size次，每个样本下[70,4]抽样20个sku在4个位置上的概率
        table = prob_table[i]  # [70,4]
        table = np.transpose(table)  # [4,70]
        table = np.exp(table * temperature)
        
        result = sample_set(table, k)  # [20,5-1]
        res.append(np.array(result))
#         for r in result:
#             count += 1
#             if (is_comb(r)):
#                 comb += 1
#                 if (is_right(r)):
#                     right += 1
#         set_count += 1
    #print(right/float(set_count), comb/float(count), set_count, count)
    print(res)
    return np.array(res)  # [bs, K, GEN_RANK_NUM-1] -> [3,20,4]

In [140]:
k_list = sample_and_test(a, 0.4)

table-shape (4, 7)
k=2
sample_one,table_shape (4, 7)
shifted_probabilities.shape: [ 0.19405188 -0.08494239 -0.08405548 -0.06243745 -0.09993549  0.16672917
 -0.02941025]
argpartition: -0.0999354873751869
table[i:,l],i=0,l=4: [0. 0. 0. 0.]
sample_one [[0.16288725 0.15630504 0.12183808 0.14837754 0.         0.13001874
  0.14002634]
 [1.34220993 1.3488912  1.14428542 1.42924673 0.         1.15245047
  1.35612176]
 [1.36457153 1.09138365 1.47885639 1.29855518 0.         1.3825336
  1.37271001]
 [1.23920415 1.04607555 1.15422115 1.26094725 0.         1.11965432
  1.09693566]] 4
shifted_probabilities.shape: [ 0.02966043 -0.12757765 -0.00296883  0.01610776  0.16740429 -0.04616066
 -0.03646533]
argpartition: -0.12757764845833952
table[i:,l],i=1,l=1: [0. 0. 0.]
sample_one [[0.16288725 0.15630504 0.12183808 0.14837754 0.         0.13001874
  0.14002634]
 [0.17267135 0.         0.14720895 0.18386838 0.         0.14825936
  0.17446107]
 [1.36457153 0.         1.47885639 1.29855518 0.         1.3825

In [24]:
k_list.shape

(3, 20, 4)

In [185]:
output

array([[[0.01763196, 0.78493079, 0.44077008, 0.31330443],
        [0.28858395, 0.15289352, 0.08271253, 0.19082091],
        [0.45205545, 0.77749992, 0.55354924, 0.72441754],
        [0.67238919, 0.24628798, 0.25232916, 0.87105074],
        [0.15843012, 0.92330014, 0.29877641, 0.22490664],
        [0.45062772, 0.31396809, 0.05024206, 0.83325644],
        [0.24008112, 0.01941202, 0.46343551, 0.38801083],
        [0.98868054, 0.4237579 , 0.0206898 , 0.75944223],
        [0.96388179, 0.20924011, 0.23936311, 0.1406516 ],
        [0.90577862, 0.22551548, 0.43431037, 0.09053341]],

       [[0.2577584 , 0.37867233, 0.10181423, 0.57026609],
        [0.39858533, 0.0453737 , 0.29448609, 0.81484825],
        [0.87736571, 0.77326876, 0.01139093, 0.36366011],
        [0.45654379, 0.55435786, 0.47245195, 0.67826014],
        [0.16794107, 0.43807598, 0.31894522, 0.04191488],
        [0.40214315, 0.08277325, 0.62191263, 0.05529954],
        [0.27910872, 0.20666079, 0.16087407, 0.06769208],
        [0.5

In [188]:
emb = tf.nn.embedding_lookup(output,k_list)
emb

<tf.Tensor 'embedding_lookup_1/Identity:0' shape=(3, 20, 4, 10, 4) dtype=float64>

In [187]:
emb_lookup = sess.run(emb)

InvalidArgumentError: indices[0,0,0] = 9 is not in [0, 3)
	 [[node embedding_lookup (defined at d:\python\anaconda3\envs\tf15\lib\site-packages\tensorflow_core\python\framework\ops.py:1748) ]]

Original stack trace for 'embedding_lookup':
  File "d:\python\anaconda3\envs\tf15\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "d:\python\anaconda3\envs\tf15\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "d:\python\anaconda3\envs\tf15\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "d:\python\anaconda3\envs\tf15\lib\site-packages\traitlets\config\application.py", line 664, in launch_instance
    app.start()
  File "d:\python\anaconda3\envs\tf15\lib\site-packages\ipykernel\kernelapp.py", line 597, in start
    self.io_loop.start()
  File "d:\python\anaconda3\envs\tf15\lib\site-packages\tornado\platform\asyncio.py", line 149, in start
    self.asyncio_loop.run_forever()
  File "d:\pyt

In [176]:
def tensor_sample(prob_table_tensor,temperature,k):
    res=tf.py_func(sample_and_test,[prob_table_tensor,temperature,k],tf.int64)
    return res

In [177]:
sess = tf.Session()

In [178]:
ts=tensor_sample(tf.convert_to_tensor(output),tf.convert_to_tensor(0.4),tf.convert_to_tensor(20))

In [179]:
ts=tensor_sample(output,0.4,3)

In [180]:
qweq= sess.run(ts)
qweq

table-shape (4, 10)
sample_one,table_shape (4, 10)
sample_one [[0.08126751 0.09057037 0.09669056 0.10559904 0.08597578 0.09663535
  0.08883014 0.11984099 0.11865811 0.        ]
 [1.36885185 1.06306624 1.36478918 1.10353117 1.44675058 1.13381408
  1.00779503 1.1847161  1.08729835 0.        ]
 [1.19280543 1.03363841 1.24784704 1.10620105 1.12694515 1.02030012
  1.20366878 1.00831026 1.10047868 0.        ]
 [1.13351313 1.07931692 1.33611616 1.41682762 1.09413343 1.39556951
  1.16789658 1.35496673 1.05787337 0.        ]] 9
sample_one [[0.08126751 0.09057037 0.09669056 0.10559904 0.08597578 0.09663535
  0.08883014 0.11984099 0.11865811 0.        ]
 [0.12720947 0.09879235 0.12683192 0.10255282 0.13444872 0.10536706
  0.09365592 0.11009746 0.         0.        ]
 [1.19280543 1.03363841 1.24784704 1.10620105 1.12694515 1.02030012
  1.20366878 1.00831026 0.         0.        ]
 [1.13351313 1.07931692 1.33611616 1.41682762 1.09413343 1.39556951
  1.16789658 1.35496673 0.         0.        ]] 8
s

array([[[9, 8, 2, 5],
        [8, 5, 2, 1],
        [9, 4, 0, 2]],

       [[6, 5, 8, 3],
        [2, 6, 3, 0],
        [7, 6, 8, 3]],

       [[8, 1, 4, 0],
        [3, 7, 2, 5],
        [0, 1, 3, 6]]], dtype=int64)

In [182]:
qweq.shape

(3, 3, 4)